#### Desafio Raízen | ANP Fuel Sales ETL Test
# RAPHAELA LEAL MAIA DE CASTRO


# Requisitos

- [X] Ser capaz de abrir e/ou converter o arquivo XLM para iniciar o tratamento.
- [ ] Desenvolver uma pipeline ETL com as colunas:  
>  - [ ] Vendas de combustíveis derivados de petróleo por UF e produto
>  - [ ] Vendas de diesel por UF e tipo
- [X] Armazenar os dados de acordo com o schema a seguir:

> |Column	     |    Type    |
  |:---------- |-----------:|
  |year_month  |	date      |
  |uf          | 	string    |
  |product	   |  string    |
  |unit	       |  string    |
  |volume	     |  double    |
  |created_at	 |  timestamp |

  - [ ]  Adicionar etapas para checar se a pipeline extrai os dados de maneira consistente em relação aos dados consolidados nas tabelas brutas. 
 

[Github com link do dataset](https://github.com/raizen-analytics/data-engineering-test/raw/master/assets/vendas-combustiveis-m3.xls)

# Instalação de bibliotecas


In [ ]:
# A biblioteca Cloud Storage permitirá a conexão entre o PySpark, o Pandas e o Bucket que está no Cloud Storage.

!pip install -q gcsfs

In [ ]:
!pip list |grep xlrd

xlrd                          1.1.0


In [ ]:
# O código a seguir foi comentado após o upgrade para evitar erros.

#!pip install --upgrade xlrd==1.2.0

In [ ]:
# Instalação do Pandera para validação de schema no Pandas.

!pip install -q pandera

     |████████████████████████████████| 197 kB 3.2 MB/s 


In [ ]:
# Instalação do pacote libreoffice (uma espécie de pacote office para Linux) para realizar a conversão do arquivo xls para xlsx.

!sudo apt-get install --no-install-recommends libreoffice

# Importação de bibliotecas

In [ ]:
#Importação das bibliotecas Cloud Storage. 
# A library 'os' serve para o python acessar o sistema operacional.

from google.cloud import storage
import os 

In [ ]:
# Importar a autenticação de usuários registrados no projeto, diretamente.
# Este é um método nativo da Google, muito seguro para autenticação, pois apenas usuários autorizados na plataforma gcp conseguem acessar. 
# Por não utilizar chaves, também não corre o risco de essas chaves "vazarem".

from google.colab import auth
auth.authenticate_user()

In [ ]:
# Importação Pandas, Numpy e Pandera.

import pandas as pd
import numpy as np
import pandera as pa

In [ ]:
import xlrd

In [ ]:
# Importação da library Subprocess, que serve para abrir o LibreOffice por dentro do Python (abrir um programa via Python).

import subprocess

In [ ]:
from datetime import datetime

# Configuração de autenticação para o bucket

```
Na GCP, foi criado um novo bucket (nomeado "datalake-raizen"). O dataset original foi enviado a esse datalake.
Não foi necessário criar uma service account, pois toda a autenticação de acesso é realizada por meio do Google Auth 
e somente usuários autorizados na plataforma gcp conseguem acessar, elevando a segurança do projeto.

```




# Client para o bucket

In [ ]:
# Códigos de acesso ao bucket para criar um dataframe no PySpark (client).
# O cliente é do Python, e também servirá para que o Pandas acesso os dados no bucket.

# Criação de variável para receber o nome do bucket ('bucket').
# Criação de uma variável para receber o caminho do arquivo ('path').

client = storage.Client()
bucket = client.get_bucket('datalake-raizen')
path = 'gs://datalake-raizen/brutos/vendas-combustiveis-m3.xls'

# Download do arquivo

In [ ]:
# A finalidade da função a seguir é baixar o arquivo a partir de um bucket. 
# Este código é encontrado na documentação oficial do Google, pronto para uso.

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your GCS object
    # source_blob_name = "storage-object-name"

    # The path to which the file should be downloaded
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    # Construct a client side representation of a blob.
    # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
    # any content from Google Cloud Storage. As we don't need additional data,
    # using `Bucket.blob` is preferred here.
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Downloaded storage object {} from bucket {} to local file {}.".format(
            source_blob_name, bucket_name, destination_file_name
        )
    )


In [ ]:
# Utilização da função anterior para baixar o arquivo diretamente do bucket criado.

download_blob('datalake-raizen', 'brutos/vendas-combustiveis-m3.xls', 'vendas-combustiveis-m3.xls')

Downloaded storage object brutos/vendas-combustiveis-m3.xls from bucket datalake-raizen to local file vendas-combustiveis-m3.xls.


# Transformação do arquivo para XLSX

In [ ]:
# 'proc' chama a execução do programa: ele recebe um subprocessso e inicializa e o Libreoffice, 
# que abre sem frontend, apenas para rodar a conversão do arquivo.
# Em 'try', o subprocess vai tentar abrir o comando. Se funcionar, ele joga o arquivo em outs e deixa o erro vazio. Senão, a informação 
# vai para o erro, e o outs fica vazio.
# Caso não rode, o subprocess vai tentar novamente sem o timeout (por meio do except). 

proc = subprocess.Popen(['libreoffice', '--headless', '--convert-to', 'xlsx', 'vendas-combustiveis-m3.xls', '--outdir', './converted/'])
try:
  outs, errs = proc.communicate(timeout=30)
except TimeoutExpired:
  proc.kill()
  outs, errs = proc.communicate()

# As documentações a seguir foram utilizadas para entender como utilizar o Subprocess e o LibreOffice. 
# https://www.digitalocean.com/community/tutorials/how-to-use-subprocess-to-run-external-programs-in-python-3
# https://docs.python.org/3/library/subprocess.html
# https://ourcodeworld.com/articles/read/867/how-to-convert-a-word-file-to-pdf-docx-to-pdf-in-libreoffice-with-the-cli-in-ubuntu-2004

# Criação de DFs com Pandas

In [ ]:
# Criação dos DFs para cada uma das sheets (abas) do arquivo, exceto a primeira aba (0), que precisará de tratamento especial, posteriormente.

df_s1 = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=1)
df_s2 = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=2)
df_s3 = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=3)
df_s4 = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=4)
df_s5 = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=5)
df_s6 = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=6)

In [ ]:
# Criação dos DFs para cada uma das sheets (abas) do arquivo, exceto a primeira aba (0), que precisará de tratamento especial, posteriormente.

"""
def nomear_df(sheet_name):
  df_s = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=sheet_name) 
  return df_s

for i in range(1,6):
  nomear_df(i)
"""

"\ndef nomear_df(sheet_name):\n  df_s = pd.read_excel('converted/vendas-combustiveis-m3.xlsx', sheet_name=sheet_name) \n  return df_s\n\nfor i in range(1,6):\n  nomear_df(i)\n"

# Visualização dos DFs

In [ ]:
# S1: combustíveis.

df_s1.head(2)

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,GASOLINA C (m3),2000,REGIÃO NORTE,RONDÔNIA,136073.253,9563.263,11341.229,9369.746,10719.983,11165.968,12312.451,11220.97,12482.281,13591.122,11940.570,11547.576,10818.094
1,GASOLINA C (m3),2000,REGIÃO NORTE,ACRE,3358.346,40001.853,3065.758,3495.290,2946.930,3023.920,3206.930,3612.58,3264.460,3835.740,3676.571,3225.610,3289.718


In [ ]:
# S2: combustíveis.

df_s2.head(2)

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RONDÔNIA,81453.67,3517.6,3681.7,4700.67,5339.2,6166.4,6539.65,7283.7,8082.85,7902.55,9383.15,9767.4,9088.8
1,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,ACRE,1483.00,11202.0,363.0,410.00,536.0,607.0,740.00,756.0,971.00,1174.00,1240.00,1439.0,1483.0


In [ ]:
# S3: combustíveis.

df_s3.head(2)

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,GLP - Até P13 (m3),2010,REGIÃO NORTE,RONDÔNIA,74941.911133,5455.704710,5766.664795,6847.914855,6263.050725,6207.913932,6309.226449,6523.245449,6309.932971,6198.226449,5956.440217,6265.773551,6837.817029
1,GLP - Até P13 (m3),2010,REGIÃO NORTE,ACRE,2338.586957,26436.096153,2017.396739,2255.280230,1987.722826,2530.503623,2404.369329,2107.056159,2063.339710,2103.465580,2126.775362,2269.896739,2231.702899


In [ ]:
# S4: segmentos.

df_s4.head(2)

,ANO,SEGMENTO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,2012,POSTO REVENDEDOR,RONDÔNIA,1715.512161,19696.681483,1560.5,1511,1622.545,1614.1,1579,1649.5,1515.1,1796.5,1684.4,1673.5,1775.024322
1,2012,POSTO REVENDEDOR,ACRE,437,468.007876,512.003938,5725.496814,443,403,492.5,477,514.485,430,447.5,581.5,519.5


In [ ]:
# S5: segmentos.

df_s5.head(2)

,ANO,SEGMENTO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,2012,POSTO REVENDEDOR,RONDÔNIA,34008.258,362364.526,26326.38,26573.85,29035.503,28283.501,29612.9,29755.7,30425.055,31783.878,32290.201,32961.8,31307.5
1,2012,POSTO REVENDEDOR,ACRE,10874,10432,10797.5,117603.75,8254,8329,9163.5,9001,9924,9513.5,10073,10864.25,10378


In [ ]:
# S6: segmentos.

df_s6.head(2)

,ANO,SEGMENTO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,2012,POSTO REVENDEDOR,RONDÔNIA,43257.342464,533566.184039,35384.625231,37674.906595,43406.974318,41205.832237,44599.337363,47693.626374,48241.369658,50118.459341,43795.973088,49097.450003,49090.287366
1,2012,POSTO REVENDEDOR,ACRE,8074.274461,6836.322718,6484.211124,78943.336155,4513.245285,4600.153819,5222.808146,5423.596136,6544.048257,7239.066353,7940.647733,8557.012064,7507.950058


# Pré Análise em Pandas

### df_s1

In [ ]:
# Visualizar número de linhas e colunas.

df_s1.shape

(4536, 17)

In [ ]:
# Checar os valores únicos da coluna.

df_s1['COMBUSTÍVEL'].unique()

array(['GASOLINA C (m3)', 'GASOLINA DE AVIAÇÃO (m3)',
       'QUEROSENE ILUMINANTE (m3)', 'QUEROSENE DE AVIAÇÃO (m3)',
       'ÓLEO DIESEL (m3)', 'ÓLEO COMBUSTÍVEL (m3)',
       'ETANOL HIDRATADO (m3)', 'GLP (m3)'], dtype=object)

In [ ]:
# Checar os valores únicos da coluna.

df_s1.ANO.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
# Verificar valores NULL em cada uma das coluna

df_s1.isna().sum()

COMBUSTÍVEL     0
ANO             0
REGIÃO          0
ESTADO          0
Jan            50
Fev            50
Mar            50
Abr            49
Mai            50
Jun            50
Jul            50
Ago            50
Set            50
Out            50
Nov            50
Dez            49
TOTAL          50
dtype: int64

In [ ]:
# Visualização das informações do dataset (quantidade de dados, types, null etc). 

df_s1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4536 entries, 0 to 4535
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   COMBUSTÍVEL  4536 non-null   object 
 1   ANO          4536 non-null   int64  
 2   REGIÃO       4536 non-null   object 
 3   ESTADO       4536 non-null   object 
 4   Jan          4486 non-null   float64
 5   Fev          4486 non-null   float64
 6   Mar          4486 non-null   float64
 7   Abr          4487 non-null   float64
 8   Mai          4486 non-null   float64
 9   Jun          4486 non-null   float64
 10  Jul          4486 non-null   float64
 11  Ago          4486 non-null   float64
 12  Set          4486 non-null   float64
 13  Out          4486 non-null   float64
 14  Nov          4486 non-null   float64
 15  Dez          4487 non-null   float64
 16  TOTAL        4486 non-null   float64
dtypes: float64(13), int64(1), object(3)
memory usage: 602.6+ KB


### df_s2

In [ ]:
# Visualizar número de linhas e colunas.

df_s2.shape

(1080, 17)

In [ ]:
# Checar os valores únicos da coluna.

df_s2['COMBUSTÍVEL'].unique()

array(['ÓLEO DIESEL S-10 (m3)', 'ÓLEO DIESEL S-500 (m3)',
       'ÓLEO DIESEL S-1800 (m3)', 'ÓLEO DIESEL MARÍTIMO (m3)',
       'ÓLEO DIESEL (OUTROS ) (m3)'], dtype=object)

In [ ]:
# Checar os valores únicos da coluna.

df_s2.ANO.unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
# Verificar valores NULL em cada uma das coluna

df_s2.isna().sum()

COMBUSTÍVEL     0
ANO             0
REGIÃO          0
ESTADO          0
Jan            30
Fev            30
Mar            30
Abr            30
Mai            30
Jun            30
Jul            31
Ago            32
Set            33
Out            33
Nov            33
Dez            32
TOTAL          31
dtype: int64

In [ ]:
# Visualização das informações do dataset (quantidade de dados, types, null etc). 

df_s1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4536 entries, 0 to 4535
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   COMBUSTÍVEL  4536 non-null   object 
 1   ANO          4536 non-null   int64  
 2   REGIÃO       4536 non-null   object 
 3   ESTADO       4536 non-null   object 
 4   Jan          4486 non-null   float64
 5   Fev          4486 non-null   float64
 6   Mar          4486 non-null   float64
 7   Abr          4487 non-null   float64
 8   Mai          4486 non-null   float64
 9   Jun          4486 non-null   float64
 10  Jul          4486 non-null   float64
 11  Ago          4486 non-null   float64
 12  Set          4486 non-null   float64
 13  Out          4486 non-null   float64
 14  Nov          4486 non-null   float64
 15  Dez          4487 non-null   float64
 16  TOTAL        4486 non-null   float64
dtypes: float64(13), int64(1), object(3)
memory usage: 602.6+ KB


### df_s3

In [ ]:
# Visualizar número de linhas e colunas.

df_s3.shape

(594, 17)

In [ ]:
# Checar os valores únicos da coluna.

df_s3['COMBUSTÍVEL'].unique()

array(['GLP - Até P13 (m3)', 'GLP - Outros (m3)'], dtype=object)

In [ ]:
# Checar os valores únicos da coluna.

df_s3.ANO.unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
# Verificar valores NULL em cada uma das coluna

df_s3.isna().sum()

COMBUSTÍVEL     0
ANO             0
REGIÃO          0
ESTADO          0
Jan            12
Fev            12
Mar            12
Abr            12
Mai            13
Jun            14
Jul            14
Ago            13
Set            12
Out            12
Nov            12
Dez            12
TOTAL          12
dtype: int64

In [ ]:
# Visualização das informações do dataset (quantidade de dados, types, null etc). 

df_s1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4536 entries, 0 to 4535
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   COMBUSTÍVEL  4536 non-null   object 
 1   ANO          4536 non-null   int64  
 2   REGIÃO       4536 non-null   object 
 3   ESTADO       4536 non-null   object 
 4   Jan          4486 non-null   float64
 5   Fev          4486 non-null   float64
 6   Mar          4486 non-null   float64
 7   Abr          4487 non-null   float64
 8   Mai          4486 non-null   float64
 9   Jun          4486 non-null   float64
 10  Jul          4486 non-null   float64
 11  Ago          4486 non-null   float64
 12  Set          4486 non-null   float64
 13  Out          4486 non-null   float64
 14  Nov          4486 non-null   float64
 15  Dez          4487 non-null   float64
 16  TOTAL        4486 non-null   float64
dtypes: float64(13), int64(1), object(3)
memory usage: 602.6+ KB


### df_s4

In [ ]:
# Visualizar número de linhas e colunas.

df_s4.shape

(729, 16)

In [ ]:
# Checar os valores únicos da coluna.

df_s4['SEGMENTO'].unique()

array(['POSTO REVENDEDOR', 'CONSUMIDOR FINAL', 'TRR'], dtype=object)

In [ ]:
# Checar os valores únicos da coluna.

df_s4.ANO.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
# Verificar valores NULL em cada uma das coluna

df_s4.isna().sum()

ANO          0
SEGMENTO     0
ESTADO       0
Jan         12
Fev         12
Mar         12
Abr         12
Mai         12
Jun         12
Jul         12
Ago         13
Set         13
Out         13
Nov         13
Dez         13
TOTAL       13
dtype: int64

### df_s5

In [ ]:
# Visualizar número de linhas e colunas.

df_s5.shape

(729, 16)

In [ ]:
# Checar os valores únicos da coluna.

df_s5['SEGMENTO'].unique()

array(['POSTO REVENDEDOR', 'CONSUMIDOR FINAL', 'TRR'], dtype=object)

In [ ]:
# Checar os valores únicos da coluna.

df_s5.ANO.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
# Verificar valores NULL em cada uma das coluna

df_s5.isna().sum()

ANO          0
SEGMENTO     0
ESTADO       0
Jan         12
Fev         12
Mar         12
Abr         12
Mai         12
Jun         12
Jul         12
Ago         13
Set         13
Out         13
Nov         13
Dez         13
TOTAL       13
dtype: int64

### df_s6

In [ ]:
# Visualizar número de linhas e colunas.

df_s6.shape

(729, 16)

In [ ]:
df_s6['SEGMENTO'].unique()

array(['POSTO REVENDEDOR', 'CONSUMIDOR FINAL', 'TRR'], dtype=object)

In [ ]:
df_s6.ANO.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
# Verificar valores NULL em cada uma das coluna

df_s6.isna().sum()

ANO          0
SEGMENTO     0
ESTADO       0
Jan         12
Fev         12
Mar         12
Abr         12
Mai         12
Jun         12
Jul         12
Ago         13
Set         13
Out         13
Nov         13
Dez         13
TOTAL       13
dtype: int64

In [ ]:
# Visualização das informações do dataset (quantidade de dados, types, null etc). 

df_s6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ANO       729 non-null    int64 
 1   SEGMENTO  729 non-null    object
 2   ESTADO    729 non-null    object
 3   Jan       717 non-null    object
 4   Fev       717 non-null    object
 5   Mar       717 non-null    object
 6   Abr       717 non-null    object
 7   Mai       717 non-null    object
 8   Jun       717 non-null    object
 9   Jul       717 non-null    object
 10  Ago       716 non-null    object
 11  Set       716 non-null    object
 12  Out       716 non-null    object
 13  Nov       716 non-null    object
 14  Dez       716 non-null    object
 15  TOTAL     716 non-null    object
dtypes: int64(1), object(15)
memory usage: 91.2+ KB


# Análise

> - As abas s1, s2 e s3 possuem valores sobre combustíveis, ano e volume de venda em m3 (metro cúbico).
- As abas s4, s5 e s6 possuem valores a respeito do segmento, ano e volume de venda em m3 (metro cúbico).
- As colunas que representam os meses são todas de floats e somente elas possuem valores null.

# To do:

> - [X] Não precisaremos das abas s4, s5 e s6.
- [X] Unir as abas s1, s2 e s3 em um único DF.
- [X] Renomear os nomes das colunas.
- [X] Renomear os valores de algumas colunas.
- [X] Tratar valores null, substiuindo-os por zero. 
- [X] Alterar a ordem das colunas de acordo com o schema proposto.
- [X] Dropar colunas desnecessárias para o projeto.
- [X] Checar e alterar types, se necessário.



# Tratamento

### Unir os dfs s1,s2 e s3

In [ ]:
# Para unir os dataframes s1, s2 e s3, vamos utilizar o concat.
# A variável 'frames' recebe uma lista com todos os dataframes. A variável 'df' receberá nosso resultado. 

frames = [df_s1, df_s2, df_s3]
df = pd.concat(frames)

In [ ]:
df.head()

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,GASOLINA C (m3),2000,REGIÃO NORTE,RONDÔNIA,136073.253,9563.263,11341.229,9369.746,10719.983,11165.968,12312.451,11220.970,12482.281,13591.122,11940.570,11547.576,10818.094
1,GASOLINA C (m3),2000,REGIÃO NORTE,ACRE,3358.346,40001.853,3065.758,3495.290,2946.930,3023.920,3206.930,3612.580,3264.460,3835.740,3676.571,3225.610,3289.718
2,GASOLINA C (m3),2000,REGIÃO NORTE,AMAZONAS,20766.918,21180.919,242742.352,17615.604,20258.200,18741.344,19604.023,20221.674,20792.616,19912.898,21869.338,21145.643,20633.175
3,GASOLINA C (m3),2000,REGIÃO NORTE,RORAIMA,3716.032,3200.400,3339.332,43338.929,3259.300,3636.216,3631.569,3348.416,3394.016,4078.616,3346.616,4029.900,4358.516
4,GASOLINA C (m3),2000,REGIÃO NORTE,PARÁ,29755.907,28661.951,28145.784,29294.796,359575.398,28830.479,32297.047,27310.979,29396.384,26511.009,36553.250,31807.840,31009.972


In [ ]:
# Visualizar número de linhas e colunas.

df.shape

(6210, 17)

In [ ]:
# Checar os valores únicos da coluna.

df.ANO.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [ ]:
# Checar os valores únicos da coluna.

df.COMBUSTÍVEL.unique()

array(['GASOLINA C (m3)', 'GASOLINA DE AVIAÇÃO (m3)',
       'QUEROSENE ILUMINANTE (m3)', 'QUEROSENE DE AVIAÇÃO (m3)',
       'ÓLEO DIESEL (m3)', 'ÓLEO COMBUSTÍVEL (m3)',
       'ETANOL HIDRATADO (m3)', 'GLP (m3)', 'ÓLEO DIESEL S-10 (m3)',
       'ÓLEO DIESEL S-500 (m3)', 'ÓLEO DIESEL S-1800 (m3)',
       'ÓLEO DIESEL MARÍTIMO (m3)', 'ÓLEO DIESEL (OUTROS ) (m3)',
       'GLP - Até P13 (m3)', 'GLP - Outros (m3)'], dtype=object)

### Substituir valores nulos

In [ ]:
df.count()

COMBUSTÍVEL    6210
ANO            6210
REGIÃO         6210
ESTADO         6210
Jan            6118
Fev            6118
Mar            6118
Abr            6119
Mai            6117
Jun            6116
Jul            6115
Ago            6115
Set            6115
Out            6115
Nov            6115
Dez            6117
TOTAL          6117
dtype: int64

In [ ]:
df = df.fillna(0)

In [ ]:
# Verificar valores NULL em cada uma das coluna

df.isna().sum()

COMBUSTÍVEL    0
ANO            0
REGIÃO         0
ESTADO         0
Jan            0
Fev            0
Mar            0
Abr            0
Mai            0
Jun            0
Jul            0
Ago            0
Set            0
Out            0
Nov            0
Dez            0
TOTAL          0
dtype: int64

### Renomear colunas

In [ ]:
# Renomear e traduzir as colunas utilizando o .columns, na ordem.

df.columns = ['combustivel', 'ano', 'regiao', 'estado', 'jan', 'fev', 'mar', 'abr', 'mai', 'jun',
              'jul', 'ago', 'set', 'out', 'nov', 'dez', 'total']

In [ ]:
df.head()

,combustivel,ano,regiao,estado,jan,fev,mar,abr,mai,jun,jul,ago,set,out,nov,dez,total
0,GASOLINA C (m3),2000,REGIÃO NORTE,RONDÔNIA,136073.253,9563.263,11341.229,9369.746,10719.983,11165.968,12312.451,11220.970,12482.281,13591.122,11940.570,11547.576,10818.094
1,GASOLINA C (m3),2000,REGIÃO NORTE,ACRE,3358.346,40001.853,3065.758,3495.290,2946.930,3023.920,3206.930,3612.580,3264.460,3835.740,3676.571,3225.610,3289.718
2,GASOLINA C (m3),2000,REGIÃO NORTE,AMAZONAS,20766.918,21180.919,242742.352,17615.604,20258.200,18741.344,19604.023,20221.674,20792.616,19912.898,21869.338,21145.643,20633.175
3,GASOLINA C (m3),2000,REGIÃO NORTE,RORAIMA,3716.032,3200.400,3339.332,43338.929,3259.300,3636.216,3631.569,3348.416,3394.016,4078.616,3346.616,4029.900,4358.516
4,GASOLINA C (m3),2000,REGIÃO NORTE,PARÁ,29755.907,28661.951,28145.784,29294.796,359575.398,28830.479,32297.047,27310.979,29396.384,26511.009,36553.250,31807.840,31009.972


In [ ]:
df.combustivel.unique()

array(['GASOLINA C (m3)', 'GASOLINA DE AVIAÇÃO (m3)',
       'QUEROSENE ILUMINANTE (m3)', 'QUEROSENE DE AVIAÇÃO (m3)',
       'ÓLEO DIESEL (m3)', 'ÓLEO COMBUSTÍVEL (m3)',
       'ETANOL HIDRATADO (m3)', 'GLP (m3)', 'ÓLEO DIESEL S-10 (m3)',
       'ÓLEO DIESEL S-500 (m3)', 'ÓLEO DIESEL S-1800 (m3)',
       'ÓLEO DIESEL MARÍTIMO (m3)', 'ÓLEO DIESEL (OUTROS ) (m3)',
       'GLP - Até P13 (m3)', 'GLP - Outros (m3)'], dtype=object)

### Backup

In [ ]:
bk_df = df.copy()

### Criar nova colunas 'product' e 'unit'

In [ ]:
# Criar a coluna 'product', que receberá os valores da coluna 'combustível'.
# Utilizar o rstrip para "arrancar o valor '(m3)' da coluna combustível".

df['product'] = df['combustivel'].map(lambda x: x.rstrip('(m3)'))

In [ ]:
# Criar a coluna 'unit' e fazer com que ela receba os valores m3.

df['unit'] = 'm3'

### Renomear valores das colunas 'estado' e 'product'

In [ ]:
df.estado.unique()

array(['RONDÔNIA', 'ACRE', 'AMAZONAS', 'RORAIMA', 'PARÁ', 'AMAPÁ',
       'TOCANTINS', 'MARANHÃO', 'PIAUÍ', 'CEARÁ', 'RIO GRANDE DO NORTE',
       'PARAÍBA', 'PERNAMBUCO', 'ALAGOAS', 'SERGIPE', 'BAHIA',
       'MINAS GERAIS', 'ESPÍRITO SANTO', 'RIO DE JANEIRO', 'SÃO PAULO',
       'PARANÁ', 'SANTA CATARINA', 'RIO GRANDE DO SUL',
       'MATO GROSSO DO SUL', 'MATO GROSSO', 'GOIÁS', 'DISTRITO FEDERAL'],
      dtype=object)

In [ ]:
lista_estados = ['RONDÔNIA', 'ACRE', 'AMAZONAS', 'RORAIMA', 'PARÁ', 'AMAPÁ', 'TOCANTINS', 'MARANHÃO', 'PIAUÍ', 'CEARÁ', 'RIO GRANDE DO NORTE',
                 'PARAÍBA', 'PERNAMBUCO', 'ALAGOAS', 'SERGIPE', 'BAHIA', 'MINAS GERAIS', 'ESPÍRITO SANTO', 'RIO DE JANEIRO', 'SÃO PAULO',
                 'PARANÁ', 'SANTA CATARINA', 'RIO GRANDE DO SUL', 'MATO GROSSO DO SUL', 'MATO GROSSO', 'GOIÁS', 'DISTRITO FEDERAL']

lista_uf = ['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP',
            'PR', 'SC', 'RS', 'MS', 'MT', 'GO', 'DF']        

df.estado.replace(lista_estados, lista_uf, inplace=True)       

In [ ]:
df['product'].unique()

array(['GASOLINA C ', 'GASOLINA DE AVIAÇÃO ', 'QUEROSENE ILUMINANTE ',
       'QUEROSENE DE AVIAÇÃO ', 'ÓLEO DIESEL ', 'ÓLEO COMBUSTÍVEL ',
       'ETANOL HIDRATADO ', 'GLP ', 'ÓLEO DIESEL S-10 ',
       'ÓLEO DIESEL S-500 ', 'ÓLEO DIESEL S-1800 ',
       'ÓLEO DIESEL MARÍTIMO ', 'ÓLEO DIESEL (OUTROS ) ',
       'GLP - Até P13 ', 'GLP - Outros '], dtype=object)

In [ ]:
lista_combustiveis = ['GASOLINA C ', 'GASOLINA DE AVIAÇÃO ', 'QUEROSENE ILUMINANTE ', 'QUEROSENE DE AVIAÇÃO ', 'ÓLEO DIESEL ', 'ÓLEO COMBUSTÍVEL ',
                      'ETANOL HIDRATADO ', 'GLP ', 'ÓLEO DIESEL S-10 ', 'ÓLEO DIESEL S-500 ', 'ÓLEO DIESEL S-1800 ', 'ÓLEO DIESEL MARÍTIMO ', 
                      'ÓLEO DIESEL (OUTROS ) ', 'GLP - Até P13 ', 'GLP - Outros ']

lista_combustiveis_f = ['GASOLINA C', 'GASOLINA DE AVIACAO', 'QUEROSENE ILUMINANTE', 'QUEROSENE DE AVIACAO', 'OLEO DIESEL', 'OLEO COMBUSTÍVEL',
                        'ETANOL HIDRATADO', 'GLP', 'OLEO DIESEL S-10', 'OLEO DIESEL S-500', 'OLEO DIESEL S-1800', 'OLEO DIESEL MARITIMO', 
                        'OLEO DIESEL - OUTROS', 'GLP - ATE P13', 'GLP - OUTROS']

df['product'].replace(lista_combustiveis, lista_combustiveis_f, inplace=True)          

### Dropar colunas 'combustivel', 'regiao' e 'total'

In [ ]:
df.drop(['combustivel','regiao','total'],axis=1 ,inplace=True)

In [ ]:
df.head()

,ano,estado,jan,fev,mar,abr,mai,jun,jul,ago,set,out,nov,dez,product,unit
0,2000,RO,136073.253,9563.263,11341.229,9369.746,10719.983,11165.968,12312.451,11220.970,12482.281,13591.122,11940.570,11547.576,GASOLINA C,m3
1,2000,AC,3358.346,40001.853,3065.758,3495.290,2946.930,3023.920,3206.930,3612.580,3264.460,3835.740,3676.571,3225.610,GASOLINA C,m3
2,2000,AM,20766.918,21180.919,242742.352,17615.604,20258.200,18741.344,19604.023,20221.674,20792.616,19912.898,21869.338,21145.643,GASOLINA C,m3
3,2000,RR,3716.032,3200.400,3339.332,43338.929,3259.300,3636.216,3631.569,3348.416,3394.016,4078.616,3346.616,4029.900,GASOLINA C,m3
4,2000,PA,29755.907,28661.951,28145.784,29294.796,359575.398,28830.479,32297.047,27310.979,29396.384,26511.009,36553.250,31807.840,GASOLINA C,m3


### Alterar a ordem das colunas

In [ ]:
# Realização de backup do df.

df_bkp = df.copy()

In [ ]:
# Utilização da função melt para alterar a orientação da tabela.
# id_vars é um parâmetro opcional (pode ser tupla, lista ou array N-dimensional). É a coluna utilizada para as variáveis identificadoras.

df = df.melt(id_vars=['product', 'ano', 'estado', 'unit'])

In [ ]:
# Visualização com sample para diversificar a exibição dos resultados.

df.sample(10)

,product,ano,estado,unit,variable,value
5646,GLP - OUTROS,2010,RR,m3,jan,78.088768
45410,QUEROSENE DE AVIACAO,2008,MS,m3,ago,2064.449000
62675,GASOLINA DE AVIACAO,2000,PI,m3,nov,86.295000
9969,ETANOL HIDRATADO,2013,TO,m3,fev,3510.500000
22077,ETANOL HIDRATADO,2001,RJ,m3,abr,15148.552000
49288,GLP - OUTROS,2013,AL,m3,ago,11358.780015
66801,OLEO DIESEL S-500,2014,RR,m3,nov,8571.600000
7291,GASOLINA DE AVIACAO,2019,AC,m3,fev,39.694000
63588,QUEROSENE ILUMINANTE,2013,RR,m3,nov,0.000000
55866,GLP - OUTROS,2020,RR,m3,set,0.000000


### Unificar as colunas 'ano' e 'meses' (variable)

In [ ]:
# Traduzir os demais valores utilizando listas e replace:
# Os meses serão transformados em Integer para que seja possível concatená-los com o ano.

lista_meses = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']
lista_meses_num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]  

df.variable.replace(lista_meses, lista_meses_num, inplace=True)

In [ ]:
# Criação da coluna 'year_month' e insersão dos valores das colunas 'ano' e 'meses' (variable).
# A função lambda é utilizada para forçar o datetime a exibir somente ano e mês.

df['year_month'] = df['ano'].astype(str) + '-' + df['variable'].astype(str)
df['year_month'] = pd.to_datetime(df['year_month']).apply(lambda x: x.strftime('%Y-%B')) 

In [ ]:
# Visualização com sample para diversificar a exibição dos resultados.

df.sample(10)

,product,ano,estado,unit,variable,value,year_month
74501,GLP - OUTROS,2020,PI,m3,12,865.918478,2020-December
42238,OLEO DIESEL S-500,2016,RN,m3,7,29365.050000,2016-July
43361,GLP - OUTROS,2018,DF,m3,7,4686.242754,2018-July
24079,OLEO DIESEL MARITIMO,2019,RS,m3,4,27488.000000,2019-April
58334,OLEO DIESEL,2006,SE,m3,10,21274.281000,2006-October
37955,GASOLINA DE AVIACAO,2004,PR,m3,7,5113.253000,2004-July
47334,ETANOL HIDRATADO,2017,RR,m3,8,71.000000,2017-August
41942,OLEO DIESEL S-10,2014,PB,m3,7,8172.480000,2014-July
69947,QUEROSENE ILUMINANTE,2018,ES,m3,12,0.000000,2018-December
58077,QUEROSENE DE AVIACAO,2018,RO,m3,10,2239.216000,2018-October


In [ ]:
# Por fim, podemos dropar as colunas 'ano' e 'variable', que não serão mais utilizadas.

df.drop(['ano','variable'],axis=1 ,inplace=True)

### Reordenar tabela e últimos ajustes para estar de acordo com o Schema proposto

In [ ]:
# Verificar types.

df.dtypes

product        object
estado         object
unit           object
value         float64
year_month     object
dtype: object

In [ ]:
# Fazer com que o df receba a ordem correta das colunas.

df = df[['year_month', 'uf', 'product', 'unit', 'volume']]

KeyError: ignored

In [ ]:
# Renomear colunas remanescentes, de acordo com o schema proposto.

df.rename(columns = {'estado':'uf', 'value':'volume'}, inplace = True)

In [ ]:
# Alterar o type da coluna 'volume' para double.

df['volume'] = pd.to_numeric(df['volume'])

### Criação da coluna 'created_at'

In [ ]:
# Criar a tabela 'created_at' para receber o timestamp.

dt = datetime.now()
df['created_at'] = datetime.timestamp(dt)

In [ ]:
# Visualização com sample para diversificar a exibição dos resultados.

df.sample(10)

,product,uf,unit,volume,year_month,created_at
9046,OLEO COMBUSTÍVEL,AC,m3,0.000000e+00,2000-February,1.663079e+09
67663,OLEO DIESEL MARITIMO,AC,m3,0.000000e+00,2020-November,1.663079e+09
73781,OLEO DIESEL - OUTROS,ES,m3,5.500000e+01,2019-December,1.663079e+09
13628,QUEROSENE ILUMINANTE,PR,m3,5.214450e+02,2002-March,1.663079e+09
33742,OLEO DIESEL,SP,m3,1.031306e+06,2015-June,1.663079e+09
13364,GASOLINA DE AVIACAO,DF,m3,5.861083e+01,2013-March,1.663079e+09
39040,QUEROSENE DE AVIACAO,GO,m3,2.958898e+03,2002-July,1.663079e+09
25777,GASOLINA DE AVIACAO,SP,m3,1.382437e+03,2013-May,1.663079e+09
65141,OLEO COMBUSTÍVEL,ES,m3,3.276199e+04,2007-November,1.663079e+09
64499,OLEO DIESEL,MS,m3,9.410449e+04,2004-November,1.663079e+09


# Validação do Schema

In [ ]:
# Criar o Schema de dados (validação).

schema = pa.DataFrameSchema(
      columns = {
          'year_month':pa.Column(pa.Object),
          'uf':pa.Column(pa.String),
          'product':pa.Column(pa.String),
          'unit':pa.Column(pa.String),
          'volume':pa.Column(pa.Float)
          }     
)

In [ ]:
# Verificar validação

schema.validate(df)

,product,uf,unit,volume,year_month,created_at
0,GASOLINA C,RO,m3,136073.253000,2000-January,1.663079e+09
1,GASOLINA C,AC,m3,3358.346000,2000-January,1.663079e+09
2,GASOLINA C,AM,m3,20766.918000,2000-January,1.663079e+09
3,GASOLINA C,RR,m3,3716.032000,2000-January,1.663079e+09
4,GASOLINA C,PA,m3,29755.907000,2000-January,1.663079e+09
...,...,...,...,...,...,...
74515,GLP - OUTROS,RS,m3,30176.443841,2020-December,1.663079e+09
74516,GLP - OUTROS,MS,m3,2519.914855,2020-December,1.663079e+09
74517,GLP - OUTROS,MT,m3,3263.902174,2020-December,1.663079e+09
74518,GLP - OUTROS,GO,m3,14036.737319,2020-December,1.663079e+09
